In [ ]:
from csv import excel
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import datetime
import sqlite3
import os
from datetime import datetime

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)


server = SimpleXMLRPCServer(("127.0.0.1", 8018), requestHandler=RequestHandler, allow_none=True)

DB_FILE = 'logs/log.db'

# Создаем БД если не существует
def init_db():
    os.makedirs(os.path.dirname(DB_FILE), exist_ok=True)
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            event_type TEXT NOT NULL,
            timestamp DATETIME NOT NULL,
            duration REAL
        )
    ''')
    conn.commit()
    conn.close()

init_db()

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.now()
server.register_function(now, 'now')

# Добавление записи в лог
def add_log(event_type, timestamp, duration=None):
    try:
        with sqlite3.connect(DB_FILE) as db:
            cursor = db.cursor()
        # conn = sqlite3.connect(DB_FILE)
        #     cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO logs (event_type, timestamp, duration)
                VALUES (?, ?, ?)
            ''', (event_type, timestamp, duration))
            # conn.commit()
            # conn.close()
            return True
    except sqlite3.Error as e:
        print('DB error: ', e)
        return False
server.register_function(add_log, 'add_log')

# Получение содержимого журнала с фильтрацией
def get_log(event_filter=False, start_time=False, end_time=False, min_duration=False, max_duration=False, logs_limit=False):
    try:
        with sqlite3.connect(DB_FILE) as db:
            cursor = db.cursor()
    # conn = sqlite3.connect(DB_FILE)
    # cursor = conn.cursor()

            query = 'SELECT event_type, timestamp, duration FROM logs WHERE 1=1'
            params = []

            if event_filter:
                query += ' AND event_type LIKE ?'
                params.append(f'%{event_filter}%')

            if start_time:
                query += ' AND timestamp >= ?'
                params.append(start_time)

            if end_time:
                query += ' AND timestamp <= ?'
                params.append(end_time)

            if min_duration:
                query += ' AND duration >= ?'
                params.append(min_duration)

            if max_duration:
                query += ' AND duration <= ?'
                params.append(max_duration)

            if logs_limit:
                query += 'limit ?'
                params.append(logs_limit)

            cursor.execute(query, params)
            rows = cursor.fetchall()

    # conn.close()

    # Преобразуем в список списков для XML-RPC
            return [[row[0], row[1], row[2]] for row in rows]
    except sqlite3.Error as e:
        print("Error while fetching data from sqlite:", e)
        return None


server.register_function(get_log, 'get_log')

print("Listening on port 8018...")
server.serve_forever()

Listening on port 8018...
